In [3]:
#READ THIS:!!!!!!!!
#Read Real Stock Testing TODO List.md for more information on what to do next.

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# SETTINGS

tickers = input("Enter ticker symbols separated by commas (e.g., AAPL,MSFT): ").upper().split(',')
start_date = input("Enter the start date (YYYY-MM-DD): ")
end_date = input("Enter the end date (YYYY-MM-DD): ")
short_window = int(input("Enter the window size for short-term SMA/EMA: "))
long_window = int(input("Enter the window size for long-term SMA/EMA: "))
use_ema = input("Use EMA instead of SMA? (True/False): ").strip().lower() == 'true'
starting_cash = float(input("Enter starting capital (e.g., 100000): "))
shares_per_trade = int(input("Enter number of shares per buy/sell (e.g., 100): "))
total_realized_profit = 0
ending_cash = starting_cash
all_results = []




def backtest_crossover(ticker):
    global total_realized_profit, ending_cash
   

    print(f"\nDownloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)

    if data.empty:
        print(f"No data found for {ticker}. Skipping.\n")
        return

    prices = data['Close'].squeeze()

    # Calculate moving averages
    if use_ema:
        short_ma = prices.ewm(span=short_window).mean()
        long_ma = prices.ewm(span=long_window).mean()
    else:
        short_ma = prices.rolling(window=short_window).mean()
        long_ma = prices.rolling(window=long_window).mean()

    # Generate signals: Buy, Sell, Hold
    signals = ['Hold']
    for i in range(1, len(prices)):
        if short_ma.iloc[i] > long_ma.iloc[i] and short_ma.iloc[i-1] <= long_ma.iloc[i-1]:
            signals.append('Buy')
        elif short_ma.iloc[i] < long_ma.iloc[i] and short_ma.iloc[i-1] >= long_ma.iloc[i-1]:
            signals.append('Sell')
        else:
            signals.append('Hold')

    # Simulate trades
    position = 0
    cash = ending_cash
    portfolio_values = []
    buy_signals_dates = []
    buy_signals_prices = []
    sell_signals_dates = []
    sell_signals_prices = []
    buy_trades = 0
    sell_trades = 0
    profit = 0
    cost_basis = 0

    for i in range(len(prices)):
        price = prices.iloc[i]
        signal = signals[i]

        if signal == 'Buy' and position == 0:
            cost = price * shares_per_trade
            if cash >= cost:
                cash -= cost
                position = shares_per_trade
                cost_basis = price
                buy_trades += 1
                buy_signals_dates.append(prices.index[i])
                buy_signals_prices.append(price)
            else:
                # Not enough cash to buy
                signals[i] = 'Hold'

        elif signal == 'Sell' and position > 0:
            cash += price * position
            profit += (price - cost_basis) * position
            position = 0
            sell_trades += 1
            sell_signals_dates.append(prices.index[i])
            sell_signals_prices.append(price)

        # Track portfolio value each day
        portfolio_value = cash + position * price
        portfolio_values.append(portfolio_value)

    # Finalize if holding position at the end
    if position > 0:
        final_price = prices.iloc[-1]
        cash += final_price * position
        profit += (final_price - cost_basis) * position
        sell_signals_dates.append(prices.index[-1])
        sell_signals_prices.append(final_price)
        position = 0
        sell_trades += 1
        portfolio_values[-1] = cash
    total_realized_profit += profit
    ending_cash = cash

    roi_percent = (profit / (shares_per_trade * cost_basis)) * 100 if cost_basis > 0 else 0

    all_results.append({
        'Ticker': ticker,
        'Buy Trades': buy_trades,
        'Sell Trades': sell_trades,
        'Ending Cash': cash,
        'Realized Profit': profit,
        'ROI (%)': roi_percent
    })

    # Plotting
    plt.figure(figsize=(14, 6))
    plt.plot(prices.index, prices, label='Price')
    plt.plot(prices.index, short_ma, label=f'Short {"EMA" if use_ema else "SMA"} ({short_window})', linestyle='--')
    plt.plot(prices.index, long_ma, label=f'Long {"EMA" if use_ema else "SMA"} ({long_window})', linestyle='--')

    # Plot buy signals
    plt.scatter(buy_signals_dates, buy_signals_prices, marker='^', color='green', label='Buy Signal', s=100)
    # Plot sell signals
    plt.scatter(sell_signals_dates, sell_signals_prices, marker='v', color='red', label='Sell Signal', s=100)

    plt.title(f"{ticker} {'EMA' if use_ema else 'SMA'} Crossover Strategy")
    plt.xlabel('Date')
    plt.ylabel('Price ($)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Print trade summary for this ticker
    print(f"\nSummary of trades for {ticker}:")
    print(f"Buy Trades: {buy_trades}")
    print(f"Sell Trades: {sell_trades}")
    print(f"Ending Cash: ${cash:,.2f}")
    print(f"Realized Profit: ${profit:,.2f}")



# Run strategy for each ticker
for ticker in tickers:
    backtest_crossover(ticker)

# Final summary
results_df = pd.DataFrame(all_results)
print("\nAll strategies completed.")
print("\nFinal Summary for all tickers:")
print(results_df.to_string(index=False))
print(f"\nTotal Realized Profit Across All Tickers: ${total_realized_profit:,.2f}")
print(f"Ending Cash After All Trades: ${ending_cash:,.2f}")
print("Thank you for using the SMA/EMA Crossover Strategy Tester!")

ModuleNotFoundError: No module named 'matplotlib'